<a href="https://colab.research.google.com/github/AnshulH/NLP-DL-Group2/blob/fnn/NLP_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/NLP-DL-Group2/
!cd /content && git clone -b fnn https://github.com/AnshulH/NLP-DL-Group2

Cloning into 'NLP-DL-Group2'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 54 (delta 1), reused 6 (delta 1), pack-reused 46
Unpacking objects: 100% (54/54), 30.20 MiB | 3.62 MiB/s, done.
Updating files: 100% (14/14), done.


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os

def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt') as f:
            for line in f:
                line = line.replace('\n','')
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        vocab.append('<unk>')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID, temp[t][1]]
            
                    
    with open(file_name,'rt') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(int(wID))
                
    return [vocab,words,corpus]

class FFNN(nn.Module):
    def __init__(self, vocab, words, d_model, d_hidden, dropout):
        super().__init__() 
    
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.dropout = dropout
        self.embeds = nn.Embedding(self.vocab_size, d_model)
        self.linear1 = nn.Linear(50, 128)
        self.nonLinear1 = nn.ReLU()
        self.linear2 = nn.Linear(128, 256)
        self.nonLinear2 = nn.ReLU()
        self.linear3 = nn.Linear(256, 2)
        self.logits = nn.Softmax()

    def forward(self, src):
        # print(src)
        embeds = self.embeds(src[0:50])
        # print(embeds)
        # src = src.type(torch.LongTensor)
        # print(type(src[0]))
        output = self.linear1(embeds)
        output = self.nonLinear2(output)
        output = self.linear2(output)
        output = self.nonLinear2(output)
        output = self.linear3(output)
        output = self.logits(output)
        return output

#          {add code to implement the FFNN}
        # return self.net(embeds)
                
    def init_weights(self):
#          {perform initializations}
        pass
        
        
class LSTM(nn.Module):
    def __init__(self,vocab,words,d_model,d_hidden,n_layers,dropout_rate):
        super().__init__()
        
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.n_layers = n_layers
        self.d_hidden = d_hidden
        self.d_model = d_model
        self.embeds = nn.Embedding(self.vocab_size, d_model)
#          {perform other initializations needed for the LSTM}
        
    def forward(self,src,h):
        embeds = self.dropout(self.embeds(src))
#          {add code to implement the LSTM}     
        pass
        return [preds,h]
    
    def init_weights(self):
#          {perform initializations}
        pass
        
    
    def detach_hidden(self, hidden):
#          {needed for training...}
        pass
        return [hidden, cell]  
    
def main():
    
    parser = argparse.ArgumentParser()
    parser.add_argument('-d_model', type=int, default=100)
    parser.add_argument('-d_hidden', type=int, default=100)
    parser.add_argument('-n_layers', type=int, default=2)
    parser.add_argument('-batch_size', type=int, default=20)
    parser.add_argument('-seq_len', type=int, default=30)
    parser.add_argument('-printevery', type=int, default=5000)
    parser.add_argument('-window', type=int, default=3)
    parser.add_argument('-epochs', type=int, default=20)
    parser.add_argument('-lr', type=float, default=0.0001)
    parser.add_argument('-dropout', type=int, default=0.35)
    parser.add_argument('-clip', type=int, default=2.0)
    parser.add_argument('-model', type=str,default='LSTM')
    parser.add_argument('-savename', type=str,default='lstm')
    parser.add_argument('-loadname', type=str)
    parser.add_argument('-trainname', type=str, default='/content/NLP-DL-Group2/hw#1/mix.train.txt')
    parser.add_argument('-validname', type=str, default='/content/NLP-DL-Group2/hw#1/mix.valid.txt')
    parser.add_argument('-testname', type=str, default='/content/NLP-DL-Group2/hw#1/mix.test.txt')
    parser.add_argument("-f", required=False)
    # print(parser)
    # params = vars(parser.parse_args())
    params = {
        'd_model': 100,
        'd_hidden': 100,
        'n_layers': 2,
        'batch_size': 20,
        'seq_len': 30,
        'printevery': 5000,
        'window': 3,
        'epochs': 20,
        'lr': 0.0001,
        'dropout': 0.35,
        'clip': 2.0,
        'model': 'FFNN',
        'savename': 'lstm',
        'loadname': None,
        'trainname': '/content/NLP-DL-Group2/hw#1/mix.train.txt',
        'validname': '/content/NLP-DL-Group2/hw#1/mix.valid.txt',
        'testname': '/content/NLP-DL-Group2/hw#1/mix.test.txt'
    }   

    torch.manual_seed(0)
    
    [vocab,words,train] = read_encode(params['trainname'],[],{},[],3)
    print('vocab: %d train: %d' % (len(vocab), len(train)))
    [vocab,words,test] = read_encode(params['testname'], vocab,words,[],-1)
    print('vocab: %d test: %d' % (len(vocab),len(test)))
    params['vocab_size'] = len(vocab)
    
    epochs = 10
    criterion = nn.CrossEntropyLoss()

    if params['model'] == 'FFNN':
        model = FFNN(vocab, words, 50, None, 0.1)
        for i in range(epochs):
            model.train()

            output = model(torch.tensor(train, dtype=torch.long))
            loss = criterion(output, torch.tensor([1] * 50))

            loss.backward()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
            optimizer.step()

            print("Epoch {}/{} Step {}/{} : Loss {:.4f}".format(1,1,i+1, 1,loss))


#          {add code to instantiate the model, train for K epochs and save model to disk}
        
#     if params.model == 'LSTM':
# #          {add code to instantiate the model, train for K epochs and save model to disk}

#     if params.model == 'FFNN_CLASSIFY':
# #          {add code to instantiate the model, recall model parameters and perform/learn classification}

#     if params.model == 'LSTM_CLASSIFY':
# #          {add code to instantiate the model, recall model parameters and perform/learn classification}
        
#     print(params)

    

if __name__ == "__main__":
    main()

vocab: 44223 train: 3366260
vocab: 44223 test: 441210
Epoch 1/1 Step 1/1 : Loss 0.6869


<ipython-input-16-34fb0d826e53>:105: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.logits(output)


Epoch 1/1 Step 2/1 : Loss 0.3470
Epoch 1/1 Step 3/1 : Loss 0.3136
Epoch 1/1 Step 4/1 : Loss 0.3133
Epoch 1/1 Step 5/1 : Loss 0.3133
Epoch 1/1 Step 6/1 : Loss 0.3133
Epoch 1/1 Step 7/1 : Loss 0.3133
Epoch 1/1 Step 8/1 : Loss 0.3133
Epoch 1/1 Step 9/1 : Loss 0.3133
Epoch 1/1 Step 10/1 : Loss 0.3133


In [ ]:
torch.manual_seed(0)
    
[vocab,words,train] = read_encode(params['trainname'],[],{},[],3)
print('vocab: %d train: %d' % (len(vocab), len(train)))
[vocab,words,test] = read_encode(params['testname'], vocab,words,[],-1)
print('vocab: %d test: %d' % (len(vocab),len(test)))
params['vocab_size'] = len(vocab)

vocab: 44223 train: 3366260
vocab: 44223 test: 441210


In [ ]:
epochs = 10
criterion = nn.CrossEntropyLoss()

if params['model'] == 'FFNN':
    model = FFNN(vocab, words, 50, None, 0.1)
    for i in range(epochs):
        model.train()

        output = model(torch.tensor(train, dtype=torch.long))
        loss = criterion(output, torch.tensor([1] * 50))

        loss.backward()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        optimizer.step()

        print("Epoch {}/{} Step {}/{} : Loss {:.4f}".format(1,1,i+1, 1,loss))

<ipython-input-16-34fb0d826e53>:105: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.logits(output)


Epoch 1/1 Step 1/1 : Loss 0.6760
Epoch 1/1 Step 2/1 : Loss 0.3474
Epoch 1/1 Step 3/1 : Loss 0.3135
Epoch 1/1 Step 4/1 : Loss 0.3133
Epoch 1/1 Step 5/1 : Loss 0.3133
Epoch 1/1 Step 6/1 : Loss 0.3133
Epoch 1/1 Step 7/1 : Loss 0.3133
Epoch 1/1 Step 8/1 : Loss 0.3133
Epoch 1/1 Step 9/1 : Loss 0.3133
Epoch 1/1 Step 10/1 : Loss 0.3133


In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [123]:
stop = set(stopwords.words('english') + list(string.punctuation))

In [129]:
original_file = '/content/NLP-DL-Group2/hw#1/mix.train.txt'
tokenized_file = '/content/NLP-DL-Group2/hw#1/mix_tokenized.train.txt'

with open(original_file) as input:
  with open(tokenized_file, "w") as output:
    curr_bio = []
    for line in input:
      line_stripped = line.strip()
      if line_stripped != '':
        if line_stripped == "<start_bio>":
          bio_person = [token for token in word_tokenize(next(input).lower()) if token != "="]
          
          # Adding name of person as stop word
          stop.update(bio_person)
        elif line_stripped == "<end_bio>":
          updated = ' '.join([token for token in curr_bio])
          output.write(updated)
          print(updated)
          curr_bio = []

          # Removing name of person as stop word
          stop -= set(bio_person)
          break
        else:
          curr_bio.extend([token for token in word_tokenize(line_stripped.lower()) if token not in stop])

2 october 1781 – 25 september 1860 swiss botanist born strasbourg studied medicine stuttgart bonn dissertation plants collected geofried trier 1787. sent natural history collection london natural history society belgium became plant pathologist started work taxonomist botanical gardens botanical garden strasbourg 1808 gave long lecture lepidoptera switzerland another genus pezizella 1813 latter work contains catalogue approximately 5,000 plants numbers yet calculated also wrote many plant families including olive-flower pisum excelsa russian goldenrod fruticosa pallidum old-growth laurel lagenaria saccharina died strasbourg 25 september 1860 == works == theropoda flower-plant genera swiss canton coimbatore brüssel 1813 swiss flora lombardy containing hundred new species flora caecosphaeria lutrina lymanthella sabionata epipactylia strasbourg 1816 neotropics switzerland three vols trier 1820–1823 london 2nd edition 2 vols 1838–1843 flora schleswig-holstein trier 1835–1838 flora south af